
<div style="background-color:#CCCCCC; padding:12px; border-radius:8px;">
<h1 style="color:#003366; text-align:center; margin:8px 0;">Revisión y limpieza de 3 DataFrames (TPI - Data Analytics)</h1>
<p style="text-align:center; color:#003366; margin:0;"><em>Notebook docente en castellano — nombres descriptivos en snake_case — código y documentación</em></p>
</div>



<div style="background-color:#CCCCCC; padding:10px; border-radius:6px;">
<h2 style="color:black; text-align:center; margin-top:6px;">Resumen</h2>

<p style="color:black;">
Este notebook está diseñado con finalidades pedagógicas. Revisa, normaliza y valida tres datasets contenidos en CSV:
</p>

<ul style="color:black;">
<li><code>marketing.csv</code> → variable: <code>df_marketing</code></li>
<li><code>ventas.csv</code> → variable: <code>df_ventas</code></li>
<li><code>clientes.csv</code> → variable: <code>df_clientes</code></li>
</ul>

<p style="color:black;">
Coloca los CSV en <code>./data_in/</code> o en <code>/mnt/data/</code>. El notebook busca primero en <code>./data_in/</code> y si no encuentra, usa <code>/mnt/data/</code> (útil para entornos donde los archivos están pre-subidos).
</p>

</div>


In [12]:
# Imports y configuración inicial (nombres en castellano)
import os
from pathlib import Path
import json
import unicodedata
import zipfile

import pandas as pd
import numpy as np


# Rutas: busca en ./data_in primero, si no existe usa /mnt/data

# Rutas locales

carpeta_entrada_local = Path('./data_in')
carpeta_entrada_mnt = Path('/mnt/data')
carpeta_reportes = Path('./reportes')
carpeta_limpios = carpeta_reportes / 'limpios'
carpeta_reportes.mkdir(parents=True, exist_ok=True)
carpeta_limpios.mkdir(parents=True, exist_ok=True)

# Nombres esperados de archivos
archivo_marketing = 'marketing.csv'
archivo_ventas = 'ventas.csv'
archivo_clientes = 'clientes.csv'

Configuración de rutas lista.
Coloca los CSV en ./data_in/ o en /mnt/data/ (ya están comprobadas ambas rutas).


In [13]:
# ---------- Funciones utilitarias en castellano (snake_case) ----------
def sacar_acentos(texto):
    """Elimina acentos (tildes) de un texto. Mantiene NaN intactos."""
    if pd.isna(texto):
        return texto
    texto = str(texto)
    normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join([c for c in normalizado if not unicodedata.combining(c)])

Funciones utilitarias definidas.


In [ ]:
TOKENS_VALOR_FALTANTE = {
    '', 'na', 'n/a', 'null', 'none', 'sin dato', 's/d', 'nd', '-', '--', '?', 'sin_dato', 'n/d'
}

def es_valor_faltante(valor):
    """Determina si un valor debe considerarse faltante (NaN)."""
    if pd.isna(valor):
        return True
    s = str(valor).strip().lower()
    s = sacar_acentos(s)
    return s in TOKENS_VALOR_FALTANTE

In [ ]:
# Detección de outliers (IQR)
def mascara_valores_atipicos_rango_intercuartil(serie_datos):
    serie_limpia = serie_datos.dropna().astype(float)
    if serie_limpia.shape[0] < 4:
        return pd.Series([False] * len(serie_datos), index=serie_datos.index)
    q1 = serie_limpia.quantile(0.25)
    q3 = serie_limpia.quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr
    return (serie_datos < limite_inferior) | (serie_datos > limite_superior)

In [ ]:
# Detección de outliers (Z-score)
def mascara_valores_atipicos_zscore(serie_datos, umbral=3.0):
    serie_limpia = serie_datos.dropna().astype(float)
    if serie_limpia.shape[0] < 4 or serie_limpia.std() == 0:
        return pd.Series([False] * len(serie_datos), index=serie_datos.index)
    puntaje_z = (serie_datos - serie_limpia.mean()) / serie_limpia.std()
    return puntaje_z.abs() > umbral

In [ ]:
print('Funciones utilitarias definidas.')

# limpieza y normalización

In [14]:
# ---------- Detección de problemas en un DataFrame ----------
def detectar_problemas_en_dataframe(df: pd.DataFrame):
    """Detecta problemas comunes en un DataFrame y retorna:
    - resumen: diccionario con métricas generales
    - chequeos_por_columna: dict con información por columna
    - problemas_df: DataFrame con filas problemáticas y descripción
    """
    resumen = {}
    resumen['filas'] = df.shape[0]
    resumen['columnas'] = df.shape[1]
    resumen['nulos_por_columna'] = df.isna().sum().to_dict()
    dup_mask = df.duplicated(keep=False)
    resumen['duplicados_exactos'] = int(dup_mask.sum())

    chequeos_por_columna = {}
    for col in df.columns:
        ser = df[col]
        info = {'dtype': str(ser.dtype), 'nulos': int(ser.isna().sum())}
        if ser.dtype == object or pd.api.types.is_string_dtype(ser):
            s = ser.astype(str)
            info['espacios_inicio'] = int(s.str.match(r'^\s+').sum())
            info['espacios_final'] = int(s.str.match(r'\s+$').sum())
            try:
                unique_original = set(s.dropna().unique())
                unique_lower = set(s.dropna().str.lower().unique())
                info['unique_original'] = len(unique_original)
                info['unique_lower'] = len(unique_lower)
                info['variantes_mayusculas'] = len(unique_lower) < len(unique_original)
            except Exception:
                info['unique_original'] = ser.nunique(dropna=True)
                info['unique_lower'] = None
                info['variantes_mayusculas'] = None
            try:
                unaccented = s.dropna().map(lambda x: sacar_acentos(x).lower())
                groups = unaccented.groupby(unaccented).size()
                conflicts = groups[groups > 1]
                info['grupos_variantes_acentos'] = int(conflicts.shape[0])
                ejemplos = {}
                if not conflicts.empty:
                    for val in conflicts.index[:5]:
                        originales = sorted(list(s[unaccented == val].unique())[:10])
                        ejemplos[val] = originales
                info['ejemplos_variantes_acentos'] = ejemplos
            except Exception:
                info['grupos_variantes_acentos'] = None
                info['ejemplos_variantes_acentos'] = {}
            info['tokens_aparente_faltante'] = int(s.map(lambda x: str(x).strip().lower()).map(lambda v: sacar_acentos(v) in TOKENS_VALOR_FALTANTE).sum())
            info['muestras'] = list(s.dropna().unique()[:10])
        else:
            if pd.api.types.is_numeric_dtype(ser):
                s_f = ser.dropna().astype(float)
                info['media'] = float(s_f.mean()) if not s_f.empty else None
                info['std'] = float(s_f.std()) if not s_f.empty else None
                info['min'] = float(s_f.min()) if not s_f.empty else None
                info['max'] = float(s_f.max()) if not s_f.empty else None
                if len(s_f) >= 4:
                    q1 = s_f.quantile(0.25)
                    q3 = s_f.quantile(0.75)
                    iqr = q3 - q1
                    lb = q1 - 1.5 * iqr
                    ub = q3 + 1.5 * iqr
                    out_iqr = (s_f < lb) | (s_f > ub)
                    info['outliers_iqr'] = int(out_iqr.sum())
                    info['limites_iqr'] = (float(lb), float(ub))
                else:
                    info['outliers_iqr'] = None
                    info['limites_iqr'] = None
                if len(s_f) >= 4 and s_f.std() != 0:
                    z = (s_f - s_f.mean()) / s_f.std()
                    info['outliers_z'] = int((z.abs() > 3).sum())
                else:
                    info['outliers_z'] = None
            else:
                parsed = pd.to_datetime(ser, errors='coerce', dayfirst=True)
                info['fechas_parseables'] = int(parsed.notna().sum())
                info['muestras'] = list(ser.dropna().unique()[:10])
        chequeos_por_columna[col] = info

    filas_problemas = []
    for idx, fila in df.iterrows():
        lista_problemas = []
        if dup_mask.loc[idx]:
            lista_problemas.append('duplicado_exacto')
        for col in df.columns:
            val = fila[col]
            # heuristicas textuales
            if pd.api.types.is_string_dtype(type(val)) or isinstance(val, str) or (not pd.isna(val) and not pd.api.types.is_numeric_dtype(type(val)) and str(chequeos_por_columna[col].get('dtype','')).startswith('object')):
                s = str(val)
                if s != s.strip():
                    lista_problemas.append(f'espacios_en_columna_{col}')
                if chequeos_por_columna[col].get('variantes_mayusculas'):
                    if s and s != s.lower() and s.lower() in [str(x).lower() for x in df[col].dropna().unique()]:
                        lista_problemas.append(f'inconsistencia_mayusculas_columna_{col}')
                if chequeos_por_columna[col].get('grupos_variantes_acentos') and chequeos_por_columna[col]['grupos_variantes_acentos'] > 0:
                    try:
                        un = sacar_acentos(s).lower()
                        group_vals = [x for x in chequeos_por_columna[col].get('muestras', []) if sacar_acentos(str(x)).lower() == un]
                        if group_vals and any(sacar_acentos(str(x)).lower() != sacar_acentos(s).lower() for x in group_vals):
                            lista_problemas.append(f'variantes_acentos_columna_{col}')
                    except Exception:
                        pass
                if es_valor_faltante(s):
                    lista_problemas.append(f'token_faltante_columna_{col}')
            else:
                try:
                    fval = float(val)
                    info_col = chequeos_por_columna[col]
                    limites = info_col.get('limites_iqr')
                    if limites and (fval < limites[0] or fval > limites[1]):
                        lista_problemas.append(f'outlier_iqr_columna_{col}')
                    if info_col.get('std') not in (None, 0):
                        mean = info_col.get('media')
                        std = info_col.get('std')
                        if std and abs((fval - mean) / std) > 3:
                            lista_problemas.append(f'outlier_z_columna_{col}')
                except Exception:
                    pass
        if lista_problemas:
            filas_problemas.append({'row_index': idx, 'problemas': ';'.join(sorted(set(lista_problemas))), 'muestra': json.dumps({str(c): str(fila[c]) for c in df.columns[:8]})})
    df_problemas = pd.DataFrame(filas_problemas)
    return resumen, chequeos_por_columna, df_problemas

print('Función detectar_problemas_en_dataframe cargada.')

Función detectar_problemas_en_dataframe cargada.


In [15]:
# ---------- Funciones de limpieza ----------
def aplicar_regla_columna(serie, regla):
    tipo, opts = regla if isinstance(regla, tuple) else (regla, {})
    s = serie.copy()
    if tipo == 'strip':
        s = s.map(lambda x: str(x).strip() if not pd.isna(x) else x)
    elif tipo == 'lower':
        s = s.map(lambda x: str(x).strip().lower() if not pd.isna(x) else x)
    elif tipo == 'upper':
        s = s.map(lambda x: str(x).strip().upper() if not pd.isna(x) else x)
    elif tipo == 'title':
        s = s.map(lambda x: str(x).strip().title() if not pd.isna(x) else x)
    elif tipo == 'quitar_acentos':
        s = s.map(lambda x: sacar_acentos(x).strip() if not pd.isna(x) else x)
    elif tipo == 'numeric':
        def to_num(v):
            if pd.isna(v):
                return np.nan
            t = str(v).strip()
            if opts.get('remove_non_digits', False):
                t = ''.join([c for c in t if c.isdigit() or c in '.-'])
            if opts.get('remove_thousands', False):
                t = t.replace(',', '')
            try:
                return int(float(t)) if opts.get('as_int', False) else float(t)
            except Exception:
                return np.nan
        s = s.map(to_num)
    elif tipo == 'date':
        def to_date(v):
            if pd.isna(v):
                return pd.NaT
            t = str(v).strip()
            if 'formats' in opts and opts['formats']:
                for fmt in opts['formats']:
                    try:
                        return pd.to_datetime(pd.to_datetime(t, format=fmt, errors='coerce'))
                    except Exception:
                        continue
            return pd.to_datetime(t, dayfirst=opts.get('dayfirst', True), errors='coerce')
        s = s.map(to_date)
    return s

Funciones de limpieza cargadas.


In [ ]:
def limpiar_dataframe(df, reglas_por_columna=None):
    reglas_por_columna = reglas_por_columna or {}
    df2 = df.copy()
    df2.columns = [str(c).strip() for c in df2.columns]
    for col in df2.columns:
        if col in reglas_por_columna:
            df2[col] = aplicar_regla_columna(df2[col], reglas_por_columna[col])
        else:
            if df2[col].dtype == object or pd.api.types.is_string_dtype(df2[col]):
                df2[col] = df2[col].map(lambda x: np.nan if es_valor_faltante(x) else (str(x).strip() if not pd.isna(x) else x))
    df2 = df2.drop_duplicates(keep='first').reset_index(drop=True)
    return df2

In [ ]:
print('Funciones de limpieza cargadas.')

In [16]:
# ---------- Proceso principal para los 3 CSV ----------
reglas_ejemplo_marketing = {
    'nombre': ('title', {}),
    'email': ('lower', {}),
    'fecha_registro': ('date', {'dayfirst': True, 'formats': ['%d/%m/%Y', '%Y-%m-%d']})
}
reglas_ejemplo_ventas = {
    'monto': ('numeric', {'remove_thousands': True, 'as_int': False}),
    'fecha_venta': ('date', {'dayfirst': True, 'formats': ['%d/%m/%Y', '%Y-%m-%d']})
}
reglas_ejemplo_clientes = {
    'apellido': ('title', {}),
    'dni': ('numeric', {'remove_non_digits': True, 'as_int': True})
}
reglas_por_archivo = {
    'marketing.csv': reglas_ejemplo_marketing,
    'ventas.csv': reglas_ejemplo_ventas,
    'clientes.csv': reglas_ejemplo_clientes
}

reportes_creados = []
zip_path = carpeta_limpios.parent / 'reports_dataset_tpi_v2.zip'


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:

# Función auxiliar para guardar CSVs
def guardar_csv(df, ruta):
    ruta = Path(ruta)
    ruta.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(ruta, index=False, encoding='utf-8')


In [ ]:

archivos = {
    'marketing.csv': ruta_entrada('marketing.csv'),
    'ventas.csv': ruta_entrada('ventas.csv'),
    'clientes.csv': ruta_entrada('clientes.csv')
}

for nombre_archivo, ruta_archivo in archivos.items():
    print(f'Procesando: {nombre_archivo}')
    if ruta_archivo is None:
        print(f'  - No se encontró {nombre_archivo} en ./data_in/ ni en /mnt/data/. Saltando.')
        continue
    try:
        df_original = pd.read_csv(ruta_archivo, dtype=str, keep_default_na=False, na_values=[''])
    except Exception:
        df_original = pd.read_csv(ruta_archivo, encoding='latin1', dtype=str, keep_default_na=False, na_values=[''])
    # Variables lógicas en castellano
    if nombre_archivo == 'marketing.csv':
        df_marketing = df_original.copy()
        df_actual = df_marketing
    elif nombre_archivo == 'ventas.csv':
        df_ventas = df_original.copy()
        df_actual = df_ventas
    else:
        df_clientes = df_original.copy()
        df_actual = df_clientes

    resumen_antes, chequeos_antes, problemas_antes = detectar_problemas_en_dataframe(df_actual)
    ruta_reporte_antes = carpeta_reportes / f'reporte_antes_limpieza_{nombre_archivo}.csv'
    guardar_csv(problemas_antes, ruta_reporte_antes)
    reportes_creados.append(ruta_reporte_antes)

    reglas = reglas_por_archivo.get(nombre_archivo, {})
    df_limpio = limpiar_dataframe(df_actual, reglas_por_columna=reglas)

    resumen_despues, chequeos_despues, problemas_despues = detectar_problemas_en_dataframe(df_limpio)
    ruta_reporte_despues = carpeta_reportes / f'reporte_despues_limpieza_{nombre_archivo}.csv'
    guardar_csv(problemas_despues, ruta_reporte_despues)
    reportes_creados.append(ruta_reporte_despues)

    ruta_chequeos_antes = carpeta_reportes / f'chequeos_antes_{nombre_archivo}.json'
    ruta_chequeos_despues = carpeta_reportes / f'chequeos_despues_{nombre_archivo}.json'
    with open(ruta_chequeos_antes, 'w', encoding='utf-8') as fh:
        json.dump(chequeos_antes, fh, ensure_ascii=False, indent=2)
    with open(ruta_chequeos_despues, 'w', encoding='utf-8') as fh:
        json.dump(chequeos_despues, fh, ensure_ascii=False, indent=2)
    reportes_creados.extend([ruta_chequeos_antes, ruta_chequeos_despues])

    resumen_path = carpeta_reportes / f'summary_{nombre_archivo}.json'
    resumen_guardar = {
        'archivo': nombre_archivo,
        'filas_antes': resumen_antes.get('filas'),
        'filas_despues': resumen_despues.get('filas'),
        'duplicados_antes': resumen_antes.get('duplicados_exactos'),
        'nulos_antes': resumen_antes.get('nulos_por_columna'),
        'filas_con_problemas_antes': len(problemas_antes),
        'filas_con_problemas_despues': len(problemas_despues)
    }
    with open(resumen_path, 'w', encoding='utf-8') as fh:
        json.dump(resumen_guardar, fh, ensure_ascii=False, indent=2)
    reportes_creados.append(resumen_path)

    ruta_cleaned = carpeta_limpios / f'cleaned_{nombre_archivo}'
    guardar_csv(df_limpio, ruta_cleaned)

# Empaquetar reportes y limpios
with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
    for p in reportes_creados:
        if Path(p).exists():
            zf.write(p, arcname=os.path.join('reports', Path(p).name))
    for p in carpeta_limpios.glob('cleaned_*.csv'):
        zf.write(p, arcname=os.path.join('limpios', p.name))

print('\nProceso finalizado. Revisa la carpeta reportes para los CSV y JSON generados.')


<div style="background-color:#CCCCCC; padding:10px; border-radius:6px;">
<h2 style="color:black; text-align:center;">Notas pedagógicas y siguientes pasos</h2>

<p style="color:black;">- Revisa los archivos <code>reporte_antes_limpieza_<archivo>.csv</code> para ver ejemplos por fila y decidir reglas adicionales.</p>
<p style="color:black;">- Ajusta <code>reglas_por_archivo</code> según los nombres reales de las columnas en tus CSV.</p>
<p style="color:black;">- Si querés que ejecute el notebook aquí y muestre resultados, subí los 3 CSV a <code>./data_in/</code> o confirma que están en <code>/mnt/data/</code>.</p>
</div>
